# Recommender System

## 1. Current Work Scope
### Customized Jaccard Index

In [2]:
skill = ["C", "Python", "Java", "R", "Unity"]
skillFilter = ["C", "Java", "Python"]

skill = [x.upper() for x in skill]
skillFilter = [x.upper() for x in skillFilter]

skill = set(skill).intersection(skillFilter)

intersection = len(list(set(skill).intersection(skillFilter)))
union = (len(set(skill)) + len(set(skillFilter))) - intersection

jaccard_index = float(intersection) / union
print(jaccard_index)

1.0


In [7]:
lang = ["Indonesian", "Javanese", "English", "Japanese", "Mandarin"]
langFilter = ["Indonesian", "English"]

lang = [x.upper() for x in lang]
langFilter = [x.upper() for x in langFilter]

lang = set(lang).intersection(langFilter)

intersection = len(list(set(lang).intersection(langFilter)))
union = (len(set(lang)) + len(set(langFilter))) - intersection

jaccard_index = float(intersection) / union
print(jaccard_index)

1.0


In [1]:
# Index with Exactly Age
age = [20]
ageFilter = [20]

age = set(age).intersection(ageFilter)

intersection = len(list(set(age).intersection(ageFilter)))
union = (len(set(age)) + len(set(ageFilter))) - intersection

jaccard_index = float(intersection) / union
print(jaccard_index)

1.0


In [4]:
# Index with Range age
age = 19
ageFilter = [23, 27]
tolerance = 5

if ageFilter[0] <= age <= ageFilter[1]:
    index = 1
elif age < ageFilter[0] and age >= ageFilter[0] - tolerance:
    index = (age - (ageFilter[0]-(tolerance+1))) / (ageFilter[0] - (ageFilter[0]-(tolerance+1)))
elif age > ageFilter[1] and age <= ageFilter[1] + tolerance:
    index = ((ageFilter[1]+(tolerance+1)) - age) / ((ageFilter[1]+(tolerance+1)) - ageFilter[1])
else:
    index = 0

print(f"Age: {age}")
print(f"Range Filter: {ageFilter[0]}-{ageFilter[1]}")
print(f"Range Tolerance: {ageFilter[0]-tolerance}-{ageFilter[1]+tolerance}")
print(f"Index: {index}")

Age: 19
Range Filter: 23-27
Range Tolerance: 18-32
Indeks: 0.3333333333333333
